In [1]:
import pandas as pd
import dspy
from typing import List, Dict

attributes = [
    'user_id',
    'Name of the Festival',
    'Date or Time of Year',
    'Season of the festival',
    'Location',
    'Historical Origins',
    'Cultural or Religious Significance',
    'Activities and Rituals',
    'Cuisine',
    'Attire and Decorations',
    'Music and Dance',
    'Participants',
    'Duration',
    'Preparation',
    'Impact on Society',
    'Modern Adaptations',
    'Language and Expressions',
    'Public Holidays',
    'Mythological or Historical Figures',
    'Inclusivity',
    'Sustainability Practices',
    'Emotional Significance'
]

festival_data = pd.DataFrame(columns=attributes)

festival_data

,user_id,Name of the Festival,Date or Time of Year,Season of the festival,Location,Historical Origins,Cultural or Religious Significance,Activities and Rituals,Cuisine,Attire and Decorations,...,Duration,Preparation,Impact on Society,Modern Adaptations,Language and Expressions,Public Holidays,Mythological or Historical Figures,Inclusivity,Sustainability Practices,Emotional Significance


In [18]:
import dspy
from typing import List, Dict
import pandas as pd

class GenerateResponse(dspy.Signature):
    """Generate natural conversational responses while collecting festival information."""
    
    context = dspy.InputField(desc="Previous conversation history")
    collected_attributes = dspy.InputField(desc="Dictionary of already collected attributes")
    missing_attributes = dspy.InputField(desc="List of attributes still needed")
    user_message = dspy.InputField()
    response = dspy.OutputField(desc="Natural conversational response that helps gather missing information")

class ExtractFestivalInfo(dspy.Signature):
    """Extract festival-related information from conversation."""
    
    conversation = dspy.InputField(desc="Complete conversation history")
    attributes_to_extract = dspy.InputField(desc="List of attributes to look for")
    extracted_attributes = dspy.OutputField(desc="Dictionary mapping attributes to extracted values")

class RAGFestivalBot(dspy.Module):
    def __init__(self, attributes: List[str]):
        super().__init__()
        self.attributes = attributes
        self.conversation_history = []
        self.collected_info = {}
        
        # Initialize DSPy modules
        self.generate_response = dspy.ChainOfThought(GenerateResponse)
        self.extract_info = dspy.ChainOfThought(ExtractFestivalInfo)
    
    def forward(self, user_message: str) -> Dict:
        try:
            # Initialize missing_attributes at the start
            missing_attributes = [attr for attr in self.attributes 
                                if attr not in self.collected_info or not self.collected_info[attr]]
            
            # Add user message to conversation history
            self.conversation_history.append(f"User: {user_message}")
            
            # Generate response using conversation history
            response_pred = self.generate_response(
                context="\n".join(self.conversation_history[-5:]),
                collected_attributes=str(self.collected_info),
                missing_attributes=str(missing_attributes),
                user_message=user_message
            )
            
            # Add bot response to conversation history
            self.conversation_history.append(f"Bot: {response_pred.response}")
            
            # Extract information from conversation
            extraction_pred = self.extract_info(
                conversation="\n".join(self.conversation_history[-2:]),
                attributes_to_extract=missing_attributes
            )
            
            # Update collected information
            if hasattr(extraction_pred, 'extracted_attributes'):
                extracted_info = extraction_pred.extracted_attributes
                if isinstance(extracted_info, dict):
                    self.collected_info.update(extracted_info)
            
            return {
                'response': response_pred.response,
                'extracted_info': self.collected_info,
                'missing_attributes': missing_attributes
            }
            
        except Exception as e:
            print(f"Error in processing: {str(e)}")
            missing_attributes = [attr for attr in self.attributes 
                                if attr not in self.collected_info or not self.collected_info[attr]]
            return {
                'response': "I apologize, but I encountered an error. Could you please rephrase that?",
                'extracted_info': self.collected_info,
                'missing_attributes': missing_attributes
            }

class FestivalDataCollector:
    def __init__(self, df: pd.DataFrame):
        self.df = df
        
    def add_festival_data(self, user_id: str, festival_info: Dict):
        """Add or update festival information for a user"""
        if user_id not in self.df['user_id'].values:
            new_row = {col: None for col in self.df.columns}
            new_row['user_id'] = user_id
            new_row.update(festival_info)
            self.df.loc[len(self.df)] = new_row
        else:
            row_idx = self.df[self.df['user_id'] == user_id].index[0]
            for key, value in festival_info.items():
                if key in self.df.columns:
                    self.df.at[row_idx, key] = value

def initialize_festival_bot(attributes: List[str], df: pd.DataFrame):
    """Initialize the festival chatbot with the given attributes"""
    chatbot = RAGFestivalBot(attributes)
    data_collector = FestivalDataCollector(df)
    return chatbot, data_collector

def chat_session(chatbot, data_collector, user_id: str, user_message: str):
    """Handle a single chat interaction"""
    try:
        # Process message and get response
        result = chatbot.forward(user_message)
        
        # Update database with any new information
        if result['extracted_info']:
            data_collector.add_festival_data(user_id, result['extracted_info'])
        
        return result['response']
    except Exception as e:
        print(f"Error in chat session: {str(e)}")
        return "I apologize, but I encountered an error. Please try again."

In [23]:
import dspy
from dotenv import load_dotenv

load_dotenv()

llm = dspy.LM(model="together_ai/meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo")

In [24]:
colbertv2_wiki17_abstracts = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')
dspy.settings.configure(lm=llm, rm=colbertv2_wiki17_abstracts)

chatbot, data_collector = initialize_festival_bot(
    attributes,
    festival_data
)

# Start a conversation
response, extracted_info = chat_session(
    chatbot,
    data_collector,
    "user123",
    "I want to tell you about pongal festival!"
)

print(f"Bot: {response}")


Bot: That's great! I'd love to learn more about Pongal festival. Can you tell me a bit about what Pongal is, when it's celebrated, and where it's typically observed?


In [25]:
response = chat_session(
    chatbot, data_collector, 
    "user123", 
    "It is widely celebrated in India especially in Tamil Nadu. Every tamil speaking person celebrate this festival"
)

In [27]:
print(f"Bot: {response}")


Bot: ("That's great to know that Pongal is widely celebrated in Tamil Nadu! Can you tell me when Pongal is typically celebrated? Is it during a specific month or season?", {})
